In [15]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
%pip list | grep json

fastjsonschema                    2.18.0
json5                             0.9.14
jsonpointer                       2.4
jsons                             1.6.3
jsonschema                        4.19.1
jsonschema-specifications         2023.7.1
pysimdjson                        5.0.2
python-json-logger                2.0.7
python-lsp-jsonrpc                1.1.1
ujson                             5.8.0
Note: you may need to restart the kernel to use updated packages.


In [17]:
import argparse
import copy
import csv
import decimal
import typing
from collections import Counter
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Set, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import simdjson as json
from IPython.display import display
from pycocotools.coco import COCO
from pycocotools.helpers import CocoClassDistHelper

In [18]:
%ls -lah "/home/gbiamby/proj/geo-llm-ret/lib/neg_refer_llm/dataset/refer_seg/refcocog/"

total 96M
drwxr-xr-x  2 gbiamby users    5 Nov 20  2016  ./
drwxr-xr-x 17 gbiamby users   26 Oct 31 05:41  ../
-rw-r--r--  1 gbiamby users 119M Feb 11  2016  instances.json
-rw-r--r--  1 gbiamby users  33M Feb 16  2016 'refs(google).p'
-rw-r--r--  1 gbiamby users  33M Nov 20  2016 'refs(umd).p'


In [19]:
REFSEG_DIR = Path("/home/gbiamby/proj/geo-llm-ret/lib/neg_refer_llm/dataset/refer_seg/")
# dataset_json = Path(REFSEG_DIR / "R-refcocog/instances.json")
# coco_dist = CocoClassDistHelper(dataset_json)


### refcocog AKA G-Ref has two versions, sort of
While we were collecting our dataset, we learned that Tamara Berg had independently applied her ReferIt game [27] to the MSCOCO dataset to generate expressions for 50,000 objects from 19,994 images. She kindly shared her data (named as UNC-Ref-COCO dataset) with us. For brevity, we call our Google Refexp dataset as G-Ref and the UNC-Ref-COCO as UNC-ref. We report results on both datasets in this paper. However, due to differences in our collection methodologies, we have found that the de- scriptions in the two overlapped datasets exhibit significant qualitative differences, with descriptions in the UNC-Ref dataset tending to be more concise and to contain less flow- ery language than our descriptions. 1 More specifically, the average lengths of expressions from our dataset and UNC- Ref are 8.43 and 3.61 respectively. And the size of the word dictionaries (keeping only words appearing more than 3 times) from our dataset and UNC-Ref are 4849 and 2890 respectively. See Figure 3 for some visual comparisons

In [31]:
pd.options.display.float_format = "{:,}".format
pd.set_option("display.max_colwidth", None)


def get_property_details(obj):
    if isinstance(obj, dict):
        first_value = list(obj.values())[0]
        if isinstance(first_value, list):
            first_item = first_value[0]
            if isinstance(first_item, dict):
                return "dict->list->dict:" + str(first_item.keys())
            else:
                return (
                    "a: dict->list->"
                    + str(type(first_item))
                    + ", lengths: "
                    + str([len(i) for i in first_item[:5]])
                )
        elif isinstance(first_value, dict):
            second_value = list(first_value.values())[0]
            if isinstance(second_value, list):
                _list = second_value
                if isinstance(_list[0], list):
                    return (
                        "b: dict->dict->list["
                        + str(type(_list[0]))
                        + "]"
                        + ", lengths: "
                        + str([len(i) for i in _list[:5]])
                    )
                else:
                    return "b: dict->dict->list[" + str(type(_list[0])) + "]"

            else:
                return (
                    "b: dict->dict->["
                    + str(type(second_value))
                    + "]"
                    + str(first_value.keys())
                )
        else:
            return "c: dict->UNKNOWN " + str(type(first_value)) + "]"
    elif isinstance(obj, list):
        first_item = obj[0]
        if isinstance(first_item, dict):
            second_value = list(first_item.values())[0]
            return (
                "d: list->dict" + str(type(second_value)) + "]" + str(first_item.keys())
            )
        else:
            return "UNKNOWN"


def get_coco_df(coco: COCO) -> pd.DataFrame:
    df_meta = pd.DataFrame(
        {
            "property": [
                "cats",
                "images",
                "anns",
                "refs_data",
                "refs",
                "img_to_refs",
                "cat_to_refs",
                "ann_to_ref",
                "ref_to_ann",
                "sents",
                "sent_to_ref",
                "sent_to_tokens",
            ],
            "count": [
                len(coco.cats),
                len(coco.imgs),
                len(coco.anns),
                len(coco.refs_data),
                len(coco.refs),
                len(coco.img_to_refs),
                len(coco.cat_to_refs),
                len(coco.ann_to_ref),
                len(coco.ref_to_ann),
                len(coco.sents),
                len(coco.sent_to_ref),
                len(coco.sent_to_tokens),
            ],
            "python_type": [
                str(type(coco.cats)),
                str(type(coco.imgs)),
                str(type(coco.anns)),
                str(type(coco.refs_data)),
                str(type(coco.refs)),
                str(type(coco.img_to_refs)),
                str(type(coco.cat_to_refs)),
                str(type(coco.ann_to_ref)),
                str(type(coco.ref_to_ann)),
                str(type(coco.sents)),
                str(type(coco.sent_to_ref)),
                str(type(coco.sent_to_tokens)),
            ],
            "dict_keys": [
                get_property_details(coco.cats),
                get_property_details(coco.imgs),
                get_property_details(coco.anns),
                get_property_details(coco.refs_data),
                get_property_details(coco.refs),
                get_property_details(coco.img_to_refs),
                get_property_details(coco.cat_to_refs),
                get_property_details(coco.ann_to_ref),
                get_property_details(coco.ref_to_ann),
                get_property_details(coco.sents),
                get_property_details(coco.sent_to_ref),
                get_property_details(coco.sent_to_tokens),
            ],
        }
    )
    # display(df_meta)
    # display(df_meta.style.format(thousands=","))


# df_meta_ref = get_coco_df(ref_coco)
# df_meta_rref = get_coco_df(rref_coco)

In [119]:
def show_sentence_counts(coco: COCO, L=3, visualize=True):
    # print("Sentence counts for ", coco.)
    sentence_counts = Counter()
    counts = []
    anns_all = []
    for idx, ref in enumerate(coco.refs_data):
        sentences: list[dict] = ref["sentences"]
        count = len(sentences)
        sentence_counts.update({count: 1})
        counts.append(
            {
                "ref_id": ref["ref_id"],
                "image_id": ref["image_id"],
                "split": ref["split"] if "split" in ref else "",
                "ann_id": ref["ann_id"],
                "category_id": ref["category_id"],
                "category": coco.cats[ref["category_id"]]["name"],
                "supercategory": coco.cats[ref["category_id"]]["supercategory"]
                if "supercategory" in coco.cats[ref["category_id"]]
                else str(coco.cats[ref["category_id"]]),
                "sent_count": len(sentences),
                "pos_sent_count": len(
                    [
                        s
                        for s in sentences
                        if ("exist" in s and s["exist"]) or "exist" not in s
                    ]
                ),
                "neg_sent_count": len(
                    [s for s in sentences if ("exist" in s and not s["exist"])]
                ),
            }
        )
    # anns
    df_anns = pd.DataFrame(coco.anns.values())
    df_anns["category"] = df_anns.category_id.apply(
        lambda cat_id: coco.cats[cat_id]["name"]
    )
    if "segmentation" in df_anns.columns:
        df_anns = df_anns.drop(columns=["segmentation"])
    if "bbox" in df_anns.columns:
        df_anns = df_anns.drop(columns=["bbox"])

    # print(sentence_counts, len(sentence_counts))
    df = pd.DataFrame(counts)

    print("pos/neg sentence_counts: ", df.pos_sent_count.sum(), df.neg_sent_count.sum())
    df_agg = pd.DataFrame(
        df.groupby(lambda x: True).agg(
            num_refs=("ref_id", "count"),
            sent_count=("sent_count", "sum"),
            total_pos_sents=("pos_sent_count", "sum"),
            total_neg_sents=("neg_sent_count", "sum"),
        )
    )
    if visualize:
        display(df_agg)
    if L >= 1:
        df_agg = pd.DataFrame(
            df.groupby(["pos_sent_count"]).agg(
                num_refs=("ref_id", "count"),
                sent_count=("sent_count", "sum"),
                total_pos_sents=("pos_sent_count", "sum"),
                total_neg_sents=("neg_sent_count", "sum"),
            )
        )
        if visualize:
            display(df_agg)
    if L >= 2:
        df_agg = pd.DataFrame(
            df.groupby(["pos_sent_count", "neg_sent_count"]).agg(
                num_refs=("ref_id", "count"),
                sent_count=("sent_count", "sum"),
                total_pos_sents=("pos_sent_count", "sum"),
                total_neg_sents=("neg_sent_count", "sum"),
            )
        )
        if visualize:
            display(df_agg)
    # display(df_agg.droplevel(axis=0, level=0).reset_index(drop=True))
    return df, df_agg.reset_index(), df_anns


# print("\nrefcoco:")
# df_refcoco, ref_recoco_agg = show_sentence_counts(ref_coco)
# print("\nR-refcoco:")
# df_rrefcoco, df_rrefcoco_agg = show_sentence_counts(rref_coco)

In [183]:
VALID_SPLITS = {
    # "R-refcoco": ["unc"],
    # "R-refcoco+": ["unc"],
    # "R-refcocog": ["umd"],
    # "refclef": ["berkeley", "unc"],
    "refcoco": ["google", "unc"],
    "refcoco+": ["unc"],
    "refcocog": [
        "google",
        "umd",
    ],  # ["google", "umd"], # google and umd seem to be identical? THey have the same aggregate stats.
    # "coconegref": ["berkeley"],
    "fprefcoco_v002": ["berkeley"],
    "fprefcoco+_v002": ["berkeley"],
    "fprefcocog_v002": ["berkeley"],
}


def build_refcoco(refseg_path: Path, dataset_name: str, split_by: str = None) -> COCO:
    assert dataset_name in VALID_SPLITS, dataset_name
    if split_by is None:
        split_by = VALID_SPLITS[dataset_name][0]
    else:
        assert split_by in VALID_SPLITS[dataset_name]
    coco = COCO(
        refseg_path / dataset_name / "instances.json",
        is_ref_dataset=True,
        dataset_name=dataset_name,
        split_by=split_by,
    )
    return coco


df_anns = []
df_refs = []
df_aggs = []
for dataset_name, split_bys in VALID_SPLITS.items():
    for split_by in split_bys:
        print("\n\n")
        print("=" * 220)
        print(f"Dataset: {dataset_name}({split_by})")
        ref_coco = build_refcoco(REFSEG_DIR, dataset_name, split_by)
        df_meta = get_coco_df(ref_coco)

        df_refcoco, df_refcoco_agg, df_ann = show_sentence_counts(
            ref_coco, L=1, visualize=False
        )
        # df_aggs
        df_refcoco_agg["dataset"] = f"{dataset_name}({split_by})"
        df_refcoco_agg["ann_count"] = len(ref_coco.anns)
        df_refcoco_agg["img_count"] = len(ref_coco.imgs)
        # Make 'dataset' the first column:
        df_refcoco_agg.insert(0, "dataset", df_refcoco_agg.pop("dataset"))
        df_aggs.append(df_refcoco_agg)

        # df_refs
        df_refcoco["dataset"] = f"{dataset_name}({split_by})"
        df_refcoco.insert(0, "dataset", df_refcoco.pop("dataset"))
        df_refs.append(df_refcoco)
        # df_anns
        df_ann["dataset"] = f"{dataset_name}({split_by})"
        df_ann.insert(0, "dataset", df_ann.pop("dataset"))
        df_anns.append(df_ann)


df_refs = pd.concat(df_refs)
df_aggs = pd.concat(df_aggs)
df_anns = pd.concat(df_anns)

display(df_anns)
display(df_refs)
display(df_aggs)




Dataset: refcoco(google)
Loading refs from '/shared/gbiamby/data/refer_seg/refcoco/refs(google).p'
Loaded 50000 refs
loading annotations into memory...
Done (t=1.18s)
creating index...
index created!
pos/neg sentence_counts:  142210 0



Dataset: refcoco(unc)
Loading refs from '/shared/gbiamby/data/refer_seg/refcoco/refs(unc).p'
Loaded 50000 refs
loading annotations into memory...
Done (t=1.03s)
creating index...
index created!
pos/neg sentence_counts:  142210 0



Dataset: refcoco+(unc)
Loading refs from '/shared/gbiamby/data/refer_seg/refcoco+/refs(unc).p'
Loaded 49856 refs
loading annotations into memory...
Done (t=4.45s)
creating index...
index created!
pos/neg sentence_counts:  141564 0



Dataset: refcocog(google)
Loading refs from '/shared/gbiamby/data/refer_seg/refcocog/refs(google).p'
Loaded 49822 refs
loading annotations into memory...
Done (t=4.83s)
creating index...
index created!
pos/neg sentence_counts:  95010 0



Dataset: refcocog(umd)
Loading refs from '/shared/gbia

,dataset,area,iscrowd,image_id,category_id,id,category
0,refcoco(google),197.30,0,98304,18,3007,dog
1,refcoco(google),"27,152.94",0,98304,63,99893,couch
2,refcoco(google),"11,087.45",0,98304,62,108703,chair
3,refcoco(google),"4,841.59",0,98304,63,115415,couch
4,refcoco(google),"19,175.43",0,98304,63,116865,couch
...,...,...,...,...,...,...,...
208955,fprefcocog_v002(berkeley),566.71,0,393207,1,1202801,person
208956,fprefcocog_v002(berkeley),729.36,0,393207,31,1836790,handbag
208957,fprefcocog_v002(berkeley),"88,909.54",0,524286,73,1099077,laptop
208958,fprefcocog_v002(berkeley),"59,106.65",0,524286,76,1116665,keyboard


,dataset,ref_id,image_id,split,ann_id,category_id,category,supercategory,sent_count,pos_sent_count,neg_sent_count
0,refcoco(google),4808,526754,train,592886,24,zebra,animal,3,3,0
1,refcoco(google),37010,150948,train,1928403,61,cake,food,3,3,0
2,refcoco(google),34500,181929,train,520916,1,person,person,3,3,0
3,refcoco(google),44471,63043,train,1819721,21,cow,animal,3,3,0
4,refcoco(google),48280,20362,train,1155578,86,vase,indoor,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...
49817,fprefcocog_v002(berkeley),49817,287303,train,135604,3,car,vehicle,2,1,1
49818,fprefcocog_v002(berkeley),49818,355159,train,2166645,1,person,person,4,2,2
49819,fprefcocog_v002(berkeley),49819,400744,train,56109,19,horse,animal,4,2,2
49820,fprefcocog_v002(berkeley),49820,82135,train,584687,22,elephant,animal,4,2,2


,dataset,pos_sent_count,num_refs,sent_count,total_pos_sents,total_neg_sents,ann_count,img_count
0,refcoco(google),1,2,2,2,0,196771,19994
1,refcoco(google),2,8159,16318,16318,0,196771,19994
2,refcoco(google),3,41473,124419,124419,0,196771,19994
3,refcoco(google),4,360,1440,1440,0,196771,19994
4,refcoco(google),5,5,25,25,0,196771,19994
5,refcoco(google),6,1,6,6,0,196771,19994
0,refcoco(unc),1,2,2,2,0,196771,19994
1,refcoco(unc),2,8159,16318,16318,0,196771,19994
2,refcoco(unc),3,41473,124419,124419,0,196771,19994
3,refcoco(unc),4,360,1440,1440,0,196771,19994


In [185]:
df_aggs.groupby("dataset").agg(
    num_refs=("num_refs", "sum"),
    sent_count=("sent_count", "sum"),
    total_pos_sents=("total_pos_sents", "sum"),
    total_neg_sents=("total_neg_sents", "sum"),
    total_ann_count=("ann_count", "min"),
    total_img_count=("img_count", "min"),
)

,num_refs,sent_count,total_pos_sents,total_neg_sents,total_ann_count,total_img_count
dataset,,,,,,
fprefcoco+_v002(berkeley),49856,276126,141564,134562,196737,19992
fprefcoco_v002(berkeley),50000,276930,142210,134720,196771,19994
fprefcocog_v002(berkeley),49822,186250,95010,91240,208960,25799
refcoco(google),50000,142210,142210,0,196771,19994
refcoco(unc),50000,142210,142210,0,196771,19994
refcoco+(unc),49856,141564,141564,0,196737,19992
refcocog(google),49822,95010,95010,0,208960,25799
refcocog(umd),49822,95010,95010,0,208960,25799


In [144]:
# display(df_anns)

In [186]:
df_img_ann = (
    df_refs.groupby(["dataset", "image_id"])
    .agg(
        img_anns=("ann_id", "nunique"),
        img_cats=("category_id", "count"),
        img_unique_cats=("category_id", "nunique"),
    )
    .sort_values(["dataset", "img_unique_cats"])
    .reset_index()
)
display(df_img_ann)

,dataset,image_id,img_anns,img_cats,img_unique_cats
0,fprefcoco+_v002(berkeley),72,2,2,1
1,fprefcoco+_v002(berkeley),110,4,4,1
2,fprefcoco+_v002(berkeley),113,3,3,1
3,fprefcoco+_v002(berkeley),144,3,3,1
4,fprefcoco+_v002(berkeley),154,2,2,1
...,...,...,...,...,...
177358,refcocog(umd),226658,6,6,5
177359,refcocog(umd),305772,5,5,5
177360,refcocog(umd),318638,8,8,5
177361,refcocog(umd),365659,7,7,5


In [187]:
display(df_refs)

,dataset,ref_id,image_id,split,ann_id,category_id,category,supercategory,sent_count,pos_sent_count,neg_sent_count
0,refcoco(google),4808,526754,train,592886,24,zebra,animal,3,3,0
1,refcoco(google),37010,150948,train,1928403,61,cake,food,3,3,0
2,refcoco(google),34500,181929,train,520916,1,person,person,3,3,0
3,refcoco(google),44471,63043,train,1819721,21,cow,animal,3,3,0
4,refcoco(google),48280,20362,train,1155578,86,vase,indoor,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...
49817,fprefcocog_v002(berkeley),49817,287303,train,135604,3,car,vehicle,2,1,1
49818,fprefcocog_v002(berkeley),49818,355159,train,2166645,1,person,person,4,2,2
49819,fprefcocog_v002(berkeley),49819,400744,train,56109,19,horse,animal,4,2,2
49820,fprefcocog_v002(berkeley),49820,82135,train,584687,22,elephant,animal,4,2,2


In [188]:
# These parts are unfinished.
# Have to gather df_imgs properly (by left joining to df_refs)k to properly count refs_per_image, sents_per_image.
# Right now the those are actually counting
df_temp = df_refs.merge(df_img_ann, how="inner", on=["dataset", "image_id"])
# display(df_temp)
df_ref_img = (
    df_temp.groupby(["dataset", "image_id"])
    .agg(
        total_anns=("ann_id", "nunique"),
        total_refs=("ref_id", "nunique"),
        total_cats=("category_id", "count"),
        total_unique_cats=("category_id", "nunique"),
        total_sents=("sent_count", "sum"),
        pos_sent_count=("pos_sent_count", "sum"),
        neg_sent_count=("neg_sent_count", "sum"),
        img_anns=("img_anns", "max"),
        img_cats=("img_cats", "max"),
        img_unique_cats=("img_unique_cats", "max"),
    )
    .sort_values(["dataset", "img_unique_cats"])
)
display(df_ref_img)
del df_temp

total_anns  total_refs  total_cats  \
dataset                   image_id                                       
fprefcoco+_v002(berkeley) 72                 2           2           2   
                          110                4           4           4   
                          113                3           3           3   
                          144                3           3           3   
                          154                2           2           2   
...                                        ...         ...         ...   
refcocog(umd)             226658             6           6           6   
                          305772             5           5           5   
                          318638             8           8           8   
                          365659             7           7           7   
                          469427             9           9           9   

                                    total_unique_cats  total_sents  \
dataset                   image_id                                   
fprefcoco+_v002(berkeley) 72                        1           12   
                          110                       1           22   
                          113                       1           17   
                          144                       1           16   
                          154                       1           12   
...                                               ...          ...   
refcocog(umd)             226658                    5           12   
                          305772                    5           10   
                          318638                    5           15   
                          365659                    5           14   
                          469427                    6           15   

                                    pos_sent_count  neg_sent_count  img_anns  \
dataset                   image_id                                             
fprefcoco+_v002(berkeley) 72                     6               6         2   
                          110                   11              11         4   
                          113                    9               8         3   
                          144                    8               8         3   
                          154                    6               6         2   
...                                            ...             ...       ...   
refcocog(umd)             226658                12               0         6   
                          305772                10               0         5   
                          318638                15               0         8   
                          365659                14               0         7   
                          469427                15               0         9   

                                    img_cats  img_unique_cats  
dataset                   image_id                             
fprefcoco+_v002(berkeley) 72               2                1  
                          110              4                1  
                          113              3                1  
                          144              3                1  
                          154              2                1  
...                                      ...              ...  
refcocog(umd)             226658           6                5  
                          305772           5                5  
                          318638           8                5  
                          365659           7                5  
                          469427           9                6  

[177363 rows x 10 columns]

In [189]:
df_summary = (
    df_ref_img.reset_index()
    .groupby(["dataset"])
    .agg(
        images=("image_id", "nunique"),
        total_refs=("total_refs", "sum"),
        refs_per_object=("total_refs", "mean"),
        total_sents=("total_sents", "sum"),
        sents_per_object=("total_sents", "mean"),
        pos_sent_count=("pos_sent_count", "sum"),
        neg_sent_count=("neg_sent_count", "sum"),
        min_cats_per_img=("img_unique_cats", "min"),
        max_cats_per_img=("img_unique_cats", "max"),
        mean_cats_per_img=("img_unique_cats", "mean"),
    )
)
pd.options.display.float_format = "{:,.2f}".format
pd.set_option("display.max_colwidth", None)

display(df_summary)

,images,total_refs,refs_per_object,total_sents,sents_per_object,pos_sent_count,neg_sent_count,min_cats_per_img,max_cats_per_img,mean_cats_per_img
dataset,,,,,,,,,,
fprefcoco+_v002(berkeley),19992,49856,2.49,276126,13.81,141564,134562,1,4,1.07
fprefcoco_v002(berkeley),19994,50000,2.50,276930,13.85,142210,134720,1,4,1.07
fprefcocog_v002(berkeley),25799,49822,1.93,186250,7.22,95010,91240,1,6,1.17
refcoco(google),19994,50000,2.50,142210,7.11,142210,0,1,4,1.07
refcoco(unc),19994,50000,2.50,142210,7.11,142210,0,1,4,1.07
refcoco+(unc),19992,49856,2.49,141564,7.08,141564,0,1,4,1.07
refcocog(google),25799,49822,1.93,95010,3.68,95010,0,1,6,1.17
refcocog(umd),25799,49822,1.93,95010,3.68,95010,0,1,6,1.17


In [190]:
df_refs[df_refs.dataset.str.startswith("fprefcoco")].groupby(["dataset", "split"]).agg(
    images=("image_id", "nunique"),
    refs_unq=("ref_id", "nunique"),
    refs=("ref_id", "count"),
    sentences=("sent_count", "sum"),
    pos_sent_count=("pos_sent_count", "sum"),
    neg_sent_count=("neg_sent_count", "sum"),
)

images  refs_unq   refs  sentences  \
dataset                   split                                       
fprefcoco+_v002(berkeley) testA     750      1975   1975      11205   
                          testB     750      1798   1798       9514   
                          train   16992     42278  42278     234445   
                          val      1500      3805   3805      20962   
fprefcoco_v002(berkeley)  test     4527      5000   5000      27599   
                          train   19213     40000  40000     221635   
                          val      4559      5000   5000      27696   
fprefcocog_v002(berkeley) train   24698     44822  44822     167550   
                          val      4650      5000   5000      18700   

                                 pos_sent_count  neg_sent_count  
dataset                   split                                  
fprefcoco+_v002(berkeley) testA            5726            5479  
                          testB            4889            4625  
                          train          120191          114254  
                          val             10758           10204  
fprefcoco_v002(berkeley)  test            14202           13397  
                          train          113762          107873  
                          val             14246           13450  
fprefcocog_v002(berkeley) train           85474           82076  
                          val              9536            9164

In [191]:
df_refs[df_refs.dataset.str.startswith("refcoco")].groupby(["dataset", "split"]).agg(
    images=("image_id", "nunique"),
    refs_unq=("ref_id", "nunique"),
    refs=("ref_id", "count"),
    sentences=("sent_count", "sum"),
    pos_sent_count=("pos_sent_count", "sum"),
    neg_sent_count=("neg_sent_count", "sum"),
)

images  refs_unq   refs  sentences  pos_sent_count  \
dataset          split                                                       
refcoco(google)  test     4527      5000   5000      14202           14202   
                 train   19213     40000  40000     113762          113762   
                 val      4559      5000   5000      14246           14246   
refcoco(unc)     testA     750      1975   1975       5657            5657   
                 testB     750      1810   1810       5095            5095   
                 train   16994     42404  42404     120624          120624   
                 val      1500      3811   3811      10834           10834   
refcoco+(unc)    testA     750      1975   1975       5726            5726   
                 testB     750      1798   1798       4889            4889   
                 train   16992     42278  42278     120191          120191   
                 val      1500      3805   3805      10758           10758   
refcocog(google) train   24698     44822  44822      85474           85474   
                 val      4650      5000   5000       9536            9536   
refcocog(umd)    test     2600      5023   5023       9602            9602   
                 train   21899     42226  42226      80512           80512   
                 val      1300      2573   2573       4896            4896   

                        neg_sent_count  
dataset          split                  
refcoco(google)  test                0  
                 train               0  
                 val                 0  
refcoco(unc)     testA               0  
                 testB               0  
                 train               0  
                 val                 0  
refcoco+(unc)    testA               0  
                 testB               0  
                 train               0  
                 val                 0  
refcocog(google) train               0  
                 val                 0  
refcocog(umd)    test                0  
                 train               0  
                 val                 0

In [192]:
from typing import Any


def compute_overlap(
    split_a: dict, split_b: dict, df_refs: pd.DataFrame, col: str
) -> dict[str, Any]:
    imgs_a = set(
        df_refs[
            (df_refs.dataset == split_a["dataset"])
            & (df_refs.split == split_a["split"])
        ][col].tolist()
    )
    imgs_b = set(
        df_refs[
            (df_refs.dataset == split_b["dataset"])
            & (df_refs.split == split_b["split"])
        ][col].tolist()
    )
    overlap = len(imgs_a.intersection(imgs_b))
    result = {
        "split_a": (split_a["dataset"], split_a["split"]),
        "split_b": (split_b["dataset"], split_b["split"]),
        f"{col}_overlap": overlap,
    }
    return result


def compute_split_overlaps(df_refs: pd.DataFrame, col: str):
    # display(df_refs.head(3))
    df_refs[(df_refs.dataset == "refcoco(google)") & (df_refs.split == "train")]

    # Get dataset/split values:
    splits = (
        df_refs.groupby(["dataset", "split"])
        .count()
        .reset_index()
        .to_dict(orient="records")
    )
    print("len splits: ", len(splits))
    # display(splits)

    # counts
    finished_splits: set[tuple] = set()
    counts = []
    for split_row in splits:
        for split_col in splits:
            current_split_pair = (
                split_row["dataset"],
                split_row["split"],
                split_col["dataset"],
                split_col["split"],
            )
            if current_split_pair in finished_splits:
                print("skipping: ", current_split_pair)
                continue
            counts.append(compute_overlap(split_row, split_col, df_refs, col))
            finished_splits.add(current_split_pair)

    # print("finished: ", finished_splits)
    df = pd.DataFrame(counts)
    # display(df)
    print("df counts before pivot: ", len(df))
    display(df.pivot(index="split_a", columns="split_b", values=f"{col}_overlap"))


compute_split_overlaps(df_refs[df_refs.dataset.str.startswith("refcoco")], "image_id")
compute_split_overlaps(df_refs[df_refs.dataset.str.startswith("refcoco")], "ref_id")

len splits:  16
df counts before pivot:  256


split_b,"(refcoco(google), test)","(refcoco(google), train)","(refcoco(google), val)","(refcoco(unc), testA)","(refcoco(unc), testB)","(refcoco(unc), train)","(refcoco(unc), val)","(refcoco+(unc), testA)","(refcoco+(unc), testB)","(refcoco+(unc), train)","(refcoco+(unc), val)","(refcocog(google), train)","(refcocog(google), val)","(refcocog(umd), test)","(refcocog(umd), train)","(refcocog(umd), val)"
split_a,,,,,,,,,,,,,,,,
"(refcoco(google), test)",4527,3989,852,178,168,3841,340,178,168,3841,340,3108,752,340,2675,158
"(refcoco(google), train)",3989,19213,4042,725,719,16341,1428,725,719,16339,1428,13903,3096,1447,12063,710
"(refcoco(google), val)",852,4042,4559,171,173,3853,362,171,173,3853,362,3180,766,349,2748,157
"(refcoco(unc), testA)",178,725,171,750,0,0,0,750,0,0,0,508,130,47,454,22
"(refcoco(unc), testB)",168,719,173,0,750,0,0,0,750,0,0,576,120,71,488,26
"(refcoco(unc), train)",3841,16341,3853,0,0,16994,0,0,0,16992,0,12345,2734,1257,10735,634
"(refcoco(unc), val)",340,1428,362,0,0,0,1500,0,0,0,1500,1079,241,123,918,62
"(refcoco+(unc), testA)",178,725,171,750,0,0,0,750,0,0,0,508,130,47,454,22
"(refcoco+(unc), testB)",168,719,173,0,750,0,0,0,750,0,0,576,120,71,488,26


len splits:  16
df counts before pivot:  256


split_b,"(refcoco(google), test)","(refcoco(google), train)","(refcoco(google), val)","(refcoco(unc), testA)","(refcoco(unc), testB)","(refcoco(unc), train)","(refcoco(unc), val)","(refcoco+(unc), testA)","(refcoco+(unc), testB)","(refcoco+(unc), train)","(refcoco+(unc), val)","(refcocog(google), train)","(refcocog(google), val)","(refcocog(umd), test)","(refcocog(umd), train)","(refcocog(umd), val)"
split_a,,,,,,,,,,,,,,,,
"(refcoco(google), test)",5000,0,0,202,185,4219,394,193,184,4247,362,4503,478,479,4244,258
"(refcoco(google), train)",0,40000,0,1580,1436,33962,3022,1593,1414,33833,3043,35838,4019,4039,33758,2060
"(refcoco(google), val)",0,0,5000,193,189,4223,395,189,200,4198,400,4481,503,505,4224,255
"(refcoco(unc), testA)",202,1580,193,1975,0,0,0,87,83,1685,118,1797,174,175,1695,101
"(refcoco(unc), testB)",185,1436,189,0,1810,0,0,87,76,1527,116,1616,188,190,1519,95
"(refcoco(unc), train)",4219,33962,4223,0,0,42404,0,1659,1520,35829,3271,37978,4277,4297,35793,2165
"(refcoco(unc), val)",394,3022,395,0,0,0,3811,142,119,3237,300,3431,361,361,3219,212
"(refcoco+(unc), testA)",193,1593,189,87,87,1659,142,1975,0,0,0,1800,175,175,1700,100
"(refcoco+(unc), testB)",184,1414,200,83,76,1520,119,0,1798,0,0,1607,189,189,1500,107


In [28]:
num_images = 50000
token_price = 3.0 / 100.0
tokens_per_image = 500
price = num_images * tokens_per_image * (1.0 / 1000) * token_price
print(price)

750.0


In [118]:
3 / 100

0.03

In [ ]:
import spacy
from Levenshtein import distance, hamming

spacy.prefer_gpu()
import spacy_transformers

nlp = spacy.load("en_core_web_trf")


def inspect_robust_ref(
    num_pos: int, num_neg: int, coco: COCO, df: pd.DataFrame, ref_index=None
):
    print("=" * 220)
    df_ref_example = df[(df.pos_sent_count == num_pos) & (df.neg_sent_count == num_neg)]
    if ref_index is None or ref_index < 0:
        ref_index = int(np.random.uniform(0, len(df_ref_example)) // 1)
        print("ref_index: ", ref_index)
    display(pd.DataFrame(df_ref_example.iloc[ref_index]))
    # print(type(df_ref_example))

    example_ref = coco.refs[df_ref_example.iloc[ref_index]["ref_id"]]

    # Show the category and ann:
    ann = coco.ref_to_ann[example_ref["ref_id"]]
    print("ann: ", ann)
    print("category: ", coco.cats[ann["category_id"]])
    # Show the ref:
    ref_display = copy.deepcopy(example_ref)
    del ref_display["sentences"]
    print("ref: ", ref_display)

    # Show sentences:
    pos_sents = [s for s in example_ref["sentences"] if s["exist"]]
    neg_sents = [s for s in example_ref["sentences"] if not s["exist"]]
    print("\npositive sentences:")
    for s in pos_sents:
        s = copy.deepcopy(s)
        del s["tokens"]
        del s["raw"]
        print(s)
        doc = nlp(s["sent"])
        print("noun chunks: ", list(doc.noun_chunks))
        # print(doc.
        spacy.displacy.render(doc, style="dep")
        spacy.displacy.render(doc, style="span")

    print("\nnegative sentences:")
    for s in neg_sents:
        s_display = copy.deepcopy(s)
        num_tokens = len(s["tokens"])
        del s_display["tokens"]
        del s_display["raw"]
        print("neg_sent: ", s_display)
        print("noun chunks: ", list(nlp(s["sent"]).noun_chunks))
        for ps in pos_sents:
            distances = [distance(ps["tokens"], s["tokens"])]
            print(
                f"\tLevenstein distances from pos_sent '{ps['sent_id']}': ",
                distances,
                [d / num_tokens for d in distances],
            )
            distances = [hamming(ps["tokens"], s["tokens"])]
            print(
                f"\tHamming    distances from pos_sent '{ps['sent_id']}': ",
                distances,
                [d / num_tokens for d in distances],
            )


## Look at an example with 2 positive and 33 negative sentences, to make sure the above counts make sense:
inspect_robust_ref(2, 10, rref_coco, df_rrefcoco)
inspect_robust_ref(2, 33, rref_coco, df_rrefcoco)

In [ ]:
inspect_robust_ref(1, 11, rref_coco, df_rrefcoco)

In [ ]:
# % pip install spacy[cuda11x,transformers]
# %python -m spacy download en_core_web_trf
# Shouldn't need this since we added transformers in the first line:
### %pip install spacy-transformers